In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

In [3]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

findspark.init()

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
from pyspark import SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

config = SparkConf().setMaster('local').setAppName('lab53')
spark = SparkSession.builder.config(conf=config).getOrCreate()
sc = spark.sparkContext

DATASET_PATH = '/content/gdrive/MyDrive/fly.csv'

In [6]:
airlinesDF = spark.read \
  .format('csv') \
  .option("header", "true") \
  .option("inferSchema","true") \
  .option("samplingRatio", "0.0001") \
  .load(DATASET_PATH)

airlinesDF.printSchema()
airlinesDF.show(5)

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

Sử dụng Column String để truy cập vào các cột Origin, Dest và Distance

In [ ]:
airlinesDF.select("Origin", "Dest", "Distance" ).show(10)

Sử dụng Columns Object để truy cập vào các cột Origin, Dest và Distance

In [ ]:
airlinesDF.select(column("Origin"), col("Dest"), airlinesDF.Distance, 'year').show(10)

Bạn cần tạo một cột mới là FlightDate = Year + Month + DayofMonth và chuyển về dạng Date

Sử dụng String Expression để thực hiện thao tác chuyển đổi dữ liệu.

In [9]:
airlinesDF.selectExpr("Origin", "Dest", "Distance", "to_date(concat(Year,Month,DayofMonth),'yyyyMMdd') as FlightDate").show(10)

+------+----+--------+----------+
|Origin|Dest|Distance|FlightDate|
+------+----+--------+----------+
|   SAN| SFO|     447|1987-10-14|
|   SAN| SFO|     447|1987-10-15|
|   SAN| SFO|     447|1987-10-17|
|   SAN| SFO|     447|1987-10-18|
|   SAN| SFO|     447|1987-10-19|
|   SAN| SFO|     447|1987-10-21|
|   SAN| SFO|     447|1987-10-22|
|   SAN| SFO|     447|1987-10-23|
|   SAN| SFO|     447|1987-10-24|
|   SAN| SFO|     447|1987-10-25|
+------+----+--------+----------+
only showing top 10 rows



Sử dụng Object Expression để thực hiện thao tác chuyển đổi dữ liệu.

In [10]:
airlinesDF.select("Origin", "Dest", "Distance", to_date(concat("Year","Month","DayofMonth"),"yyyyMMdd").alias("FlightDate")).show(10)

+------+----+--------+----------+
|Origin|Dest|Distance|FlightDate|
+------+----+--------+----------+
|   SAN| SFO|     447|1987-10-14|
|   SAN| SFO|     447|1987-10-15|
|   SAN| SFO|     447|1987-10-17|
|   SAN| SFO|     447|1987-10-18|
|   SAN| SFO|     447|1987-10-19|
|   SAN| SFO|     447|1987-10-21|
|   SAN| SFO|     447|1987-10-22|
|   SAN| SFO|     447|1987-10-23|
|   SAN| SFO|     447|1987-10-24|
|   SAN| SFO|     447|1987-10-25|
+------+----+--------+----------+
only showing top 10 rows

